In [1]:
from langchain_groq import ChatGroq


In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("who is the father of Artificial neural networks..")
print(response.content)

The father of Artificial Neural Networks (ANNs) is often considered to be Warren McCulloch and Walter Pitts, but more specifically, Warren McCulloch is often referred to as the "father of artificial neural networks" or the "father of neural networks".


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2737890/data-scientist-affs")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Data Scientist, AFFS - Job ID: 2737890 | Amazon.jobs
Skip to main contentHomeYour job applicationAmazon culture & benefitsDiversity at AmazonLocationsTeamsJob categoriesResourcesInterview tipsDisability accommodationsAbout AmazonFAQ×Data Scientist, AFFSJob ID: 2737890 | ADCI - MaharashtraApply nowDESCRIPTIONAmazon Financials Foundation Services (AFFS), a division within Amazon’s eCommerce Services Organization (eCS) is leading innovation in business systems integration and defining the future of financial accounting at Amazon scale. Our systems are advancing one of the world's most scalable, reliable, and secure e-commerce ecosystem and responsible for processing hundreds of billions of dollars in transactions, in multiple currencies and countries. We are at the center of Amazon’s key initiatives and fueling the growth of Amazon’s businesses worldwide by constantly raising the bar on the speed at which business teams can integrate with our systems.Do you love problem solving? Are you l

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Data Scientist, AFFS",
  "experience": "3+ years",
  "skills": [
    "Data querying languages (e.g. SQL)",
    "Scripting languages (e.g. Python)",
    "Statistical/mathematical software (e.g. R, SAS, Matlab, etc.)",
    "Machine learning/statistical modeling data analysis tools and techniques",
    "Python, Perl, or another scripting language",
    "Optimisation, machine learning, and statistical modeling/analysis",
    "Feature engineering of massive datasets",
    "Exploratory data analysis",
    "Model building using industry standard time Series Forecasting techniques",
    "Supervised(Linear/Logistic Regression) and UnSupervised algorithms(k means clustering, Principle Component Analysis, Market Basket analysis)"
  ],
  "description": "Amazon Financial Foundation Services is looking for a highly motivated Data Scientist to help build scalable, predictive and prescriptive business analytics & Machine Learning solutions that supports AFFS organization. You will be 

In [9]:
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Data Scientist, AFFS',
 'experience': '3+ years',
 'skills': ['Data querying languages (e.g. SQL)',
  'Scripting languages (e.g. Python)',
  'Statistical/mathematical software (e.g. R, SAS, Matlab, etc.)',
  'Machine learning/statistical modeling data analysis tools and techniques',
  'Python, Perl, or another scripting language',
  'Optimisation, machine learning, and statistical modeling/analysis',
  'Feature engineering of massive datasets',
  'Exploratory data analysis',
  'Model building using industry standard time Series Forecasting techniques',
  'Supervised(Linear/Logistic Regression) and UnSupervised algorithms(k means clustering, Principle Component Analysis, Market Basket analysis)'],
 'description': 'Amazon Financial Foundation Services is looking for a highly motivated Data Scientist to help build scalable, predictive and prescriptive business analytics & Machine Learning solutions that supports AFFS organization. You will be working with Global Stakeholders, Da

In [11]:
type(json_res)


dict

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'http

In [19]:
job

{'role': 'Data Scientist, AFFS',
 'experience': '3+ years',
 'skills': ['Data querying languages (e.g. SQL)',
  'Scripting languages (e.g. Python)',
  'Statistical/mathematical software (e.g. R, SAS, Matlab, etc.)',
  'Machine learning/statistical modeling data analysis tools and techniques',
  'Python, Perl, or another scripting language',
  'Optimisation, machine learning, and statistical modeling/analysis',
  'Feature engineering of massive datasets',
  'Exploratory data analysis',
  'Model building using industry standard time Series Forecasting techniques',
  'Supervised(Linear/Logistic Regression) and UnSupervised algorithms(k means clustering, Principle Component Analysis, Market Basket analysis)'],
 'description': 'Amazon Financial Foundation Services is looking for a highly motivated Data Scientist to help build scalable, predictive and prescriptive business analytics & Machine Learning solutions that supports AFFS organization. You will be working with Global Stakeholders, Da

In [20]:
job = json_res
job['skills']

['Data querying languages (e.g. SQL)',
 'Scripting languages (e.g. Python)',
 'Statistical/mathematical software (e.g. R, SAS, Matlab, etc.)',
 'Machine learning/statistical modeling data analysis tools and techniques',
 'Python, Perl, or another scripting language',
 'Optimisation, machine learning, and statistical modeling/analysis',
 'Feature engineering of massive datasets',
 'Exploratory data analysis',
 'Model building using industry standard time Series Forecasting techniques',
 'Supervised(Linear/Logistic Regression) and UnSupervised algorithms(k means clustering, Principle Component Analysis, Market Basket analysis)']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohammed, a business development executive at Anas Data Science. Anas Data Science is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Anas Data Science 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Anas Data Science's portfolio: {link_list}
        Remember you are Mohammed, BDE at Anas Data Science. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Amazon Financial Foundation Services

Dear Hiring Manager,

I came across the job posting for a Data Scientist at Amazon Financial Foundation Services and was impressed by the organization's commitment to building scalable, predictive, and prescriptive business analytics and Machine Learning solutions. As a Business Development Executive at Anas Data Science, I believe our team can help you achieve your goals.

At Anas Data Science, we specialize in facilitating seamless integration of business processes through automated tools. Our expertise in AI and software consulting has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team has extensive experience in data querying languages (e.g., SQL), scripting languages (e.g., Python), and statistical/mathematical software (e.g., R, SAS, Matlab). We are well-versed in machine learning/statistica